In [1]:
!pip install datasets
!pip install torch
!pip install transformers
!pip install tqdm
!pip install pandas
!pip install scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.3 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 10.0.1
    Uninstalling pyarrow-10.0.1:
      Successfully uninstalled pyarrow-10.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ibis-framework 7.1.0 requires pyarrow<15,>=2, but you have pyarrow 15.0.0 which is incompatible.


In [2]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
from datasets import load_dataset
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [3]:
dataset = load_dataset("mteb/amazon_reviews_multi")
en_reviews = dataset['train'].filter(lambda example: example['id'].startswith('en_')).shuffle(seed=42).select([i for i in range(10000)])

# бин классификация
en_reviews = en_reviews.filter(lambda example: example['label'] != 2)
en_reviews = en_reviews.map(lambda example: {'label': 0 if example['label'] in [0, 1] else 1, 'text': example['text']})


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1454: FutureWarning: The repository for mteb/amazon_reviews_multi contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mteb/amazon_reviews_multi
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `d

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Filter:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/7933 [00:00<?, ? examples/s]

In [4]:
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaForSequenceClassification.from_pretrained(model_name, num_labels=2).to('cuda')

max_length = 512
tokenized_texts = [tokenizer(review['text'], max_length=max_length, padding='max_length', truncation=True, return_tensors='pt') for review in en_reviews]
labels = torch.tensor([example['label'] for example in en_reviews]).to('cuda')

dataset = TensorDataset(torch.cat([t['input_ids'] for t in tokenized_texts]).to('cuda'),
                        torch.cat([t['attention_mask'] for t in tokenized_texts]).to('cuda'),
                        labels)

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

In [8]:
batch_size = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [9]:
def train_model(model, train_dataloader, epochs=3):
    optimizer = AdamW(model.parameters(), lr=1e-5)
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        progress_bar = tqdm(train_dataloader, desc=f'Epoch {epoch + 1}/{epochs}', leave=False)
        for batch in progress_bar:
            optimizer.zero_grad()
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            progress_bar.set_postfix({'Loss': total_loss / (progress_bar.n + 1e-12)})
        print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_dataloader)}")

train_model(model, train_dataloader)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1, Loss: 0.21873597081154356


Epoch 2, Loss: 0.12234920241577646


Epoch 3, Loss: 0.07946001917282366


In [10]:
def evaluate_model(model, test_dataloader, model_name):
    model.eval()
    predictions = []
    true_labels = []
    with torch.no_grad():
        for batch in test_dataloader:
            input_ids, attention_mask, labels = batch
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            batch_predictions = torch.argmax(logits, dim=1)
            predictions.extend(batch_predictions.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions)
    recall = recall_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    confusion_mat = confusion_matrix(true_labels, predictions)
    results = {
        'Accuracy': accuracy * 100,
        'Precision': precision * 100,
        'Recall': recall * 100,
        'F1 Score': f1 * 100,
    }
    df = pd.DataFrame(results, index=[model_name])
    print(f"Results for {model_name}:")
    print(df)
    print("Confusion Matrix:")
    print(confusion_mat)
    print("\n" + "=" * 50 + "\n")

evaluate_model(model, test_dataloader, 'RoBERTa')

Results for RoBERTa:
          Accuracy  Precision     Recall   F1 Score
RoBERTa  93.509767   95.34282  91.666667  93.468611
Confusion Matrix:
[[747  36]
 [ 67 737]]




In [1]:
imdb_dataset = load_dataset("imdb")

random_indices = torch.randperm(len(imdb_dataset['train']))[:int(0.2 * len(imdb_dataset['train']))]
imdb_reviews = imdb_dataset['train'].select(random_indices)

imdb_reviews = imdb_reviews.map(lambda example: {'label': 0 if example['label'] == 'neg' else 1, 'text': example['text']})

tokenized_imdb_texts = [tokenizer(review['text'], max_length=max_length, padding='max_length', truncation=True, return_tensors='pt') for review in imdb_reviews]
labels_imdb = torch.tensor([example['label'] for example in imdb_reviews]).to('cuda')

batch_size_imdb = 2
imdb_dataloader = DataLoader(TensorDataset(torch.cat([t['input_ids'] for t in tokenized_imdb_texts]).to('cuda'),
                                            torch.cat([t['attention_mask'] for t in tokenized_imdb_texts]).to('cuda'),
                                            labels_imdb),
                             batch_size=batch_size_imdb,
                             shuffle=False)

evaluate_model(model, imdb_dataloader, 'RoBERTa on IMDb (pre-trained on Amazon)')


NameError: name 'load_dataset' is not defined

# Сохраню модель на гугл диск

In [11]:
import os

output_dir = 'my_model'
os.makedirs(output_dir, exist_ok=True)


In [12]:
# Save tokenizer
tokenizer.save_pretrained(output_dir)

# Save model
model.save_pretrained(output_dir)


In [13]:
import shutil

shutil.make_archive(output_dir, 'zip', output_dir)


'/content/my_model.zip'

In [14]:
from google.colab import drive

drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [15]:
shutil.move(output_dir + '.zip', '/content/gdrive/MyDrive/')


'/content/gdrive/MyDrive/my_model.zip'